In [1]:
from elasticsearch import Elasticsearch
from functools import partial

# Create an elastic search engine
es = Elasticsearch(
     cloud_id="lm-datasets:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDk1N2U5ODIwZDUxNTQ0YWViMjk0MmQwNzI1NjE0OTQ2JDhkN2M0OWMyZDEzMTRiNmM4NDNhNGEwN2U4NDE5NjRl",
     api_key="T2w5Vl9ZTUIzVzE5dTlBblUtRlo6MHNBYWxhbHVTeGFodUpUOWIybkNNZw==",
     retry_on_timeout=True,
     http_compress=True,
)

# Syntactic sugar to avoid repeating index and keyword arguments
search = partial(
    es.search,
    
    index="re_pile",
    track_total_hits=True,
)

In [33]:
query = {
    "match": {"text": {"query": "Disney animation", "operator": "and"}}
}

def scroll(query, size, scroll):
    data = search(query=query, size=size, scroll=scroll)
    print("Total documents found", data["hits"]["total"])

    sid = data["_scroll_id"]
    scroll_size = len(data["hits"]["hits"])

    
    while scroll_size > 0:
        print("Processing", scroll_size, "documents")
        
        yield data["hits"]["hits"]
        
        data = es.scroll(scroll_id=sid, scroll="2m")
        sid = data["_scroll_id"]

        scroll_size = len(data["hits"]["hits"])

    es.clear_scroll(scroll_id=sid)
    

def to_json(data, filepath, mode="a"):
    import json
    data = data if isinstance(data, (list, tuple)) else [data]
    data = [json.dumps(l, sort_keys=True) for l in data]
    
    with open(filepath, mode=mode) as f:
        f.writelines(data)
        
# Sanity check
docs_iter = iter(scroll(query, 100, "10m"))
to_json(next(docs_iter), "temp.json", "w")
to_json(next(docs_iter), "temp.json")

Total documents found {'value': 82575, 'relation': 'eq'}
Processing 100 documents
Processing 100 documents
